### <u> Machine Learning</u>


Let's start with imports:

In [139]:
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_validate, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [121]:
df = pd.read_csv('combined_finished_df.csv')

In [122]:
# Declaring variables to use with train_test method
y = df['Rating']
test = df['Description']
X = df.drop(columns = ['Rating'])

In [172]:
categoricals = list(X.select_dtypes('object').columns)
categoricals = categoricals.remove('Description')
numericals = list(X.select_dtypes(exclude=['object']).columns)

In [173]:
tfidf = TfidfVectorizer(min_df=50,max_features=80000)
v = tfidf.fit_transform(text)
embedding = pd.DataFrame(v.toarray().astype(np.uint8), columns=tfidf.get_feature_names())
print(embedding.shape)

(47472, 9455)


In [174]:
features = []
for feature in numericals:
    enc = LabelEncoder()
    encoded = enc.fit(X[feature])
    encoded = enc.transform(X[feature])
    features.append(encoded)
features = np.array(features)

In [178]:
num_df = pd.DataFrame(features,columns=numericals)
final_df = pd.concat([num_df,embedding],axis=1)

In [180]:
pipeline = Pipeline(steps=[('model',LogisticRegression(random_state=42))])
"""
results = cross_validate(pipeline, 
                         X_train, 
                         y_train, 
                         return_train_score=True)
"""

'\nresults = cross_validate(pipeline, \n                         X_train, \n                         y_train, \n                         return_train_score=True)\n'

In [182]:
grid = {'model__C': [0.5,1,50]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=grid,
                           cv=5,
                           return_train_score=True)
grid_search.fit(final_df, y)

E:\ProgramFiles\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
E:\ProgramFiles\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
E:\ProgramFiles\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the d

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('model',
                                        LogisticRegression(random_state=42))]),
             param_grid={'model__C': [0.5, 1, 50]}, return_train_score=True)

In [183]:
print(f"Average train score: {grid_search.cv_results_['mean_train_score'].mean()}")
print(f"Average Test score: {grid_search.cv_results_['mean_test_score'].mean()}")

Average train score: 0.8540402764046208
Average Test score: 0.8540402768722976


In [55]:
est = grid_search.best_estimator_ 
est = est.predict(X_test)